In [1]:
import pandas as pd
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:

model_xgb = joblib.load(r'D:\Cloudly IO\Tesla-Forcasting-Project\notebook\saved_models\model_xgb.pkl') #Load model
model_lnr = joblib.load(r'D:\Cloudly IO\Tesla-Forcasting-Project\notebook\saved_models\model_lnr.pkl') #Load model
model_nn = joblib.load(r'D:\Cloudly IO\Tesla-Forcasting-Project\notebook\saved_models\model_nn.pkl') #Load model
model_lstm = joblib.load(r'D:\Cloudly IO\Tesla-Forcasting-Project\notebook\saved_models\model_lstm.pkl') #Load model
model_gru = joblib.load(r'D:\Cloudly IO\Tesla-Forcasting-Project\notebook\saved_models\model_gru.pkl') #Load model

In [7]:
def inference(data):
    import numpy as np
    scaler_X = joblib.load(r'D:\Cloudly IO\Tesla-Forcasting-Project\notebook\saved_models\scaler_X.pkl')
    scaler_y = joblib.load(r'D:\Cloudly IO\Tesla-Forcasting-Project\notebook\saved_models\scaler_y.pkl')


    # Ensure input is 2D array
    data = np.array(data).reshape(1, -1)

    # Standardize input features
    data_scaled = scaler_X.fit_transform(data)

    # Predict and inverse scale
    pred_lnr = model_lnr.predict(data_scaled).reshape(-1, 1)
    pred_xgb = model_xgb.predict(data_scaled).reshape(-1, 1)
    pred_nn = model_nn.predict(data_scaled).reshape(-1, 1)

    # Inverse transform predictions
    output_lnr = scaler_y.inverse_transform(pred_lnr)
    output_xgb = scaler_y.inverse_transform(pred_xgb)
    output_nn = scaler_y.inverse_transform(pred_nn)

    # Print predictions
    print("Output From Linear Regression:", output_lnr[0][0])
    print("Output From XGBoost:", output_xgb[0][0])
    print("Output From Neural Network:", output_nn[0][0])


In [8]:
inference([[338, 344, 331, 335, 334]])

d:\Cloudly IO\Tesla-Forcasting-Project\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
Output From Linear Regression: 108.18950729925058
Output From XGBoost: 108.88429
Output From Neural Network: 109.2015


In [18]:
queue_data = np.array(
    [[236.86000061, 240.11999512, 234.8999939 , 237.49000549, 248.90399628],
       [236.13999939, 241.25      , 235.30000305, 240.44999695, 247.49899597],
       [238.11000061, 238.96000671, 232.03999329, 234.96000671, 245.74099731],
       [235.1000061 , 235.5       , 231.28999329, 233.94000244, 243.47399902],
       [230.57000732, 230.92999268, 225.36999512, 227.22000122, 240.0519989 ]]
)

In [19]:
import numpy as np

# Example: queue_data is initialized with a 5x5 sequence
# queue_data = X_train_RNN[2000]  # Must be shape (5, 5)

def inference_seq(new_row):
    scaler_X = joblib.load(r'D:\Cloudly IO\Tesla-Forcasting-Project\notebook\saved_models\scaler_X.pkl')
    scaler_y = joblib.load(r'D:\Cloudly IO\Tesla-Forcasting-Project\notebook\saved_models\scaler_y.pkl')
    global queue_data
    new_row.append(np.mean(queue_data[:,4]))

    # Ensure the new_row is of shape (5,)
    new_row = np.array(new_row)
    if new_row.shape != (5,):
        raise ValueError("Each input row must be of shape (5,)")

    # Append the new row and remove the first row
    queue_data = np.vstack([queue_data[1:], new_row])

    # Scale input using the same scaler as used during training
    queue_scaled = scaler_X.transform(queue_data)

    # Reshape for model input: (1, 5, 5)
    data_scaled = queue_scaled.reshape(1, 5, 5)

    # Predict
    pred_lstm_scaled = model_lstm.predict(data_scaled).reshape(-1, 1)
    pred_gru_scaled = model_gru.predict(data_scaled).reshape(-1, 1)

    # Inverse transform to get predictions in original target scale
    pred_lstm = scaler_y.inverse_transform(pred_lstm_scaled)
    pred_gru = scaler_y.inverse_transform(pred_gru_scaled)

    # Output
    print("Output From LSTM:", pred_lstm[0][0])
    print("Output From GRU :", pred_gru[0][0])


In [22]:
inference_seq([338, 344, 331, 340])

d:\Cloudly IO\Tesla-Forcasting-Project\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
Output From LSTM: 325.52597
Output From GRU : 331.7526
